In [ ]:
# # usage: python Image_Stitching [/PATH/img1] [/PATH/img2]
# # e.g.: python Image_Stitching.py ./images/q11.jpg ./images/q22.jpg

# import cv2
# import numpy as np
# import sys

# class Image_Stitching():
#     def __init__(self) :
#         self.ratio=0.85
#         self.min_match=10
#         self.sift=cv2.xfeatures2d.SIFT_create()
#         self.smoothing_window_size=800

#     def registration(self,img1,img2):
#         kp1, des1 = self.sift.detectAndCompute(img1, None)
#         kp2, des2 = self.sift.detectAndCompute(img2, None)
#         matcher = cv2.BFMatcher()
#         raw_matches = matcher.knnMatch(des1, des2, k=2)
#         good_points = []
#         good_matches=[]
#         for m1, m2 in raw_matches:
#             if m1.distance < self.ratio * m2.distance:
#                 good_points.append((m1.trainIdx, m1.queryIdx))
#                 good_matches.append([m1])
#         img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, flags=2)
#         cv2.imwrite('matching.jpg', img3)
#         if len(good_points) > self.min_match:
#             image1_kp = np.float32(
#                 [kp1[i].pt for (_, i) in good_points])
#             image2_kp = np.float32(
#                 [kp2[i].pt for (i, _) in good_points])
#             H, status = cv2.findHomography(image2_kp, image1_kp, cv2.RANSAC,5.0)
#         return H

#     def create_mask(self,img1,img2,version):
#         height_img1 = img1.shape[0]
#         width_img1 = img1.shape[1]
#         width_img2 = img2.shape[1]
#         height_panorama = height_img1
#         width_panorama = width_img1 +width_img2
#         offset = int(self.smoothing_window_size / 2)
#         barrier = img1.shape[1] - int(self.smoothing_window_size / 2)
#         mask = np.zeros((height_panorama, width_panorama))
#         if version== 'left_image':
#             mask[:, barrier - offset:barrier + offset ] = np.tile(np.linspace(1, 0, 2 * offset ).T, (height_panorama, 1))
#             mask[:, :barrier - offset] = 1
#         else:
#             mask[:, barrier - offset :barrier + offset ] = np.tile(np.linspace(0, 1, 2 * offset ).T, (height_panorama, 1))
#             mask[:, barrier + offset:] = 1
#         return cv2.merge([mask, mask, mask])

#     def blending(self,img1,img2):
#         H = self.registration(img1,img2)
#         height_img1 = img1.shape[0]
#         width_img1 = img1.shape[1]
#         width_img2 = img2.shape[1]
#         height_panorama = height_img1
#         width_panorama = width_img1 +width_img2

#         panorama1 = np.zeros((height_panorama, width_panorama, 3))
#         mask1 = self.create_mask(img1,img2,version='left_image')
#         panorama1[0:img1.shape[0], 0:img1.shape[1], :] = img1
#         panorama1 *= mask1
#         mask2 = self.create_mask(img1,img2,version='right_image')
#         panorama2 = cv2.warpPerspective(img2, H, (width_panorama, height_panorama))*mask2
#         result=panorama1+panorama2

#         rows, cols = np.where(result[:, :, 0] != 0)
#         min_row, max_row = min(rows), max(rows) + 1
#         min_col, max_col = min(cols), max(cols) + 1
#         final_result = result[min_row:max_row, min_col:max_col, :]
#         return final_result

# def main(argv1,argv2):
#     img1 = cv2.imread(argv1)
#     img2 = cv2.imread(argv2)
#     final=Image_Stitching().blending(img1,img2)
#     cv2.imwrite('panorama.jpg', final)

# if __name__ == '__main__':
#     try: 
#         main(sys.argv[1],sys.argv[2])
#     except IndexError:
#         print ("Please input two source images: ")
#         print ("For example: python Image_Stitching.py '/Users/linrl3/Desktop/picture/p1.jpg' '/Users/linrl3/Desktop/picture/p2.jpg'")
    



In [ ]:
# for iter in range(0,N):
#     Randomly select 4 feature pairs
#     Compute homography H
#     Compute inliers where SSD(p, Hp) < e
#     Keep largest set of inliers

In [13]:
import cv2
import numpy as np
import random
import sys
import matplotlib.pyplot as plt

In [2]:
CFG = {
    'ratio': 0.85,
    'min_match': 10,
    'smoothing_window_size': 800,
}

In [ ]:
def findFeatures(img):
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    
    return keypoints, descriptors

In [ ]:
def matchFeatures(kp1, kp2, desc1, desc2, img1, img2):
    matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = matcher.match(desc1, desc2)

    return matches

In [ ]:
#
# Computers a homography from 4-correspondences
#
def calculateHomography(correspondences):
    #loop through correspondences and create assemble matrix
    aList = []
    for corr in correspondences:
        p1 = np.matrix([corr.item(0), corr.item(1), 1])
        p2 = np.matrix([corr.item(2), corr.item(3), 1])

        a2 = [0, 0, 0, -p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2),
              p2.item(1) * p1.item(0), p2.item(1) * p1.item(1), p2.item(1) * p1.item(2)]
        a1 = [-p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2), 0, 0, 0,
              p2.item(0) * p1.item(0), p2.item(0) * p1.item(1), p2.item(0) * p1.item(2)]
        aList.append(a1)
        aList.append(a2)
 
    matrixA = np.matrix(aList)

    #svd composition
    u, s, v = np.linalg.svd(matrixA)

    #reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    
    return h

In [ ]:
#
#Calculate the geometric distance between estimated points and original points
#
def geometricDistance(correspondence, h):

    p1 = np.transpose(np.matrix([correspondence[0].item(0), correspondence[0].item(1), 1]))
    estimatep2 = np.dot(h, p1)
    estimatep2 = (1/estimatep2.item(2))*estimatep2

    p2 = np.transpose(np.matrix([correspondence[0].item(2), correspondence[0].item(3), 1]))
    error = p2 - estimatep2
    
    return np.linalg.norm(error)

In [ ]:
#
#Runs through ransac algorithm, creating homographies from random correspondences
#
def ransac(corr, thresh):
    maxInliers = []
    finalH = None
    for i in range(1000):
        #find 4 random points to calculate a homography
        corr1 = corr[random.randrange(0, len(corr))]
        corr2 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((corr1, corr2))
        corr3 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr3))
        corr4 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr4))

        #call the homography function on those points
        h = calculateHomography(randomFour)
        inliers = []

        for i in range(len(corr)):
            d = geometricDistance(corr[i], h)
            if d < 5:
                inliers.append(corr[i])

        if len(inliers) > len(maxInliers):
            maxInliers = inliers
            finalH = h
        print("Corr size: ", len(corr), " NumInliers: ", len(inliers), "Max inliers: ", len(maxInliers))

        if len(maxInliers) > (len(corr)*thresh):
            break
    
    return finalH, maxInliers

In [ ]:
# def registration(self,img1,img2):
#     keypoints_1, descriptor_1 = cv2.sift.detectAndCompute(img1, None)
#     keypoints_2, descriptor_2 = cv2.sift.detectAndCompute(img2, None)
    
#     matcher = cv2.BFMatcher()
#     raw_matches = matcher.knnMatch(descriptor_1, descriptor_2, k=2)
    
#     good_points = []
#     good_matches=[]
#     for m1, m2 in raw_matches:
#         if m1.distance < self.ratio * m2.distance:
#             good_points.append((m1.trainIdx, m1.queryIdx))
#             good_matches.append([m1])
    
#     img3 = cv2.drawMatchesKnn(img1, keypoints_1, img2, keypoints_2, good_matches, None, flags=2)
#     cv2.imwrite('matching.jpg', img3)
    
#     if len(good_points) > self.min_match:
#         image1_kp = np.float32(
#             [keypoints_1[i].pt for (_, i) in good_points])
#         image2_kp = np.float32(
#             [keypoints_2[i].pt for (i, _) in good_points])
#         H, status = cv2.findHomography(image2_kp, image1_kp, cv2.RANSAC, 5.0)
#     return H

In [ ]:
def create_mask(self,img1,img2,version):
    height_img1 = img1.shape[0]
    width_img1 = img1.shape[1]
    width_img2 = img2.shape[1]
    height_panorama = height_img1
    width_panorama = width_img1 + width_img2
    offset = int(self.smoothing_window_size / 2)
    barrier = img1.shape[1] - int(self.smoothing_window_size / 2)
    mask = np.zeros((height_panorama, width_panorama))
    if version== 'left_image':
        mask[:, barrier - offset:barrier + offset ] = np.tile(np.linspace(1, 0, 2 * offset ).T, (height_panorama, 1))
        mask[:, :barrier - offset] = 1
    else:
        mask[:, barrier - offset :barrier + offset ] = np.tile(np.linspace(0, 1, 2 * offset ).T, (height_panorama, 1))
        mask[:, barrier + offset:] = 1
    return cv2.merge([mask, mask, mask])

In [ ]:
def blending(self,img1,img2):

    correspondenceList = []
    if img1 is not None and img2 is not None:
        kp1, desc1 = findFeatures(img1)
        kp2, desc2 = findFeatures(img2)

        keypoints = [kp1,kp2]
        matches = matchFeatures(kp1, kp2, desc1, desc2, img1, img2)
        for match in matches:
            (x1, y1) = keypoints[0][match.queryIdx].pt
            (x2, y2) = keypoints[1][match.trainIdx].pt
            correspondenceList.append([x1, y1, x2, y2])

        corrs = np.matrix(correspondenceList)

        #run ransac algorithm
        finalH, inliers = ransac(corrs, estimation_thresh)


    #H = self.registration(img1,img2)
    height_img1 = img1.shape[0]
    width_img1 = img1.shape[1]
    width_img2 = img2.shape[1]
    height_panorama = height_img1
    width_panorama = width_img1 +width_img2

    panorama1 = np.zeros((height_panorama, width_panorama, 3))
    mask1 = self.create_mask(img1,img2,version='left_image')
    panorama1[0:img1.shape[0], 0:img1.shape[1], :] = img1
    panorama1 *= mask1
    mask2 = self.create_mask(img1,img2,version='right_image')
    panorama2 = cv2.warpPerspective(img2, finalH, (width_panorama, height_panorama))*mask2
    result=panorama1+panorama2

    rows, cols = np.where(result[:, :, 0] != 0)
    min_row, max_row = min(rows), max(rows) + 1
    min_col, max_col = min(cols), max(cols) + 1
    final_result = result[min_row:max_row, min_col:max_col, :]
    return final_result

In [ ]:
img1 = cv2.imread('./images/q11.jpg')
img2 = cv2.imread('./images/q22.jpg')

In [ ]:
final = blending(img1,img2)
cv2.imwrite('panorama.jpg', final)